Selecting files from the alignment and convert to counts

In [6]:
%%bash 
  # featurecounts from bioconda subread
# Define paths
ALIGNED_DIR="data/alignment_STAR_subset/"  
OUTPUT_DIR="data/gene_counts_DESeq"  # Define output directory
GTF_FILE="/human_genome/cleaned_Homo_sapiens.GRCh38.113.gtf"  # GTF annotation file

# Create output directory 
mkdir -p "$OUTPUT_DIR"

# Loop through BAM files
for file in ${ALIGNED_DIR}/*sortedByCoord.out.bam; do
    # Extract sample name
    SAMPLE=$(basename "$file" _Aligned.sortedByCoord.out.bam)

    # Run featureCounts
    featureCounts -T 8 \
        -a "human_genome/Homo_sapiens.GRCh38.113.gtf" \
        -o "${OUTPUT_DIR}/${SAMPLE}.counts.txt" \
        -g gene_id \
        -t exon \
        -s 0 \
        -p \
        "$file"
done



/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory

        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.8

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           ERR950158_trimmed_Aligned.sortedByCoord.out.bam  ||
||                   

merge all files into a count matrix

In [12]:
import os
import pandas as pd

# Define directory
count_dir = "data/gene_counts_DESeq/"

# List all count files
count_files = [f for f in os.listdir(count_dir) if f.endswith("trimmed.counts.txt")]

# Read and merge count files
dfs = []
for file in count_files:
    df = pd.read_csv(os.path.join(count_dir, file), sep="\t", skiprows=1, usecols=[0, 6])  # Geneid & counts
    sample_name = file.replace("trimmed.counts.txt", "")
    df.rename(columns={df.columns[1]: sample_name}, inplace=True)
    dfs.append(df)

# Merge all dataframes
merged_counts = dfs[0]
for df in dfs[1:]:
    merged_counts = merged_counts.merge(df, on="Geneid", how="outer")

# Save merged file
merged_counts.to_csv("merged_counts_matrix.csv", index=False)




Merged count matrix saved as 'merged_counts_matrix.csv'.


Run DESeq2

In [34]:

import rpy2.robjects as robjects

r_deseq_script = """
library(DESeq2)                                              

# Load count matrix
count_matrix <- read.table("data/STAR_DE_analysis/STAR_merged_counts.txt",  header = TRUE, sep = "\t", row.names=1)
#count_matrix <- count_matrix[, -1]
metadata <- read.table("data/STAR_DE_analysis/STAR_metadata.txt", header = TRUE)
count_matrix <- as.matrix(count_matrix)
mode(count_matrix) <- "numeric"


rownames(metadata) <- metadata$Sample_ID

#create the factors from diagnosis
metadata$Diagnosis <- factor(metadata$Diagnosis, levels = c("Atr", "Hp"))

samples <- rownames(metadata)

# Keep only samples present in both metadata and count matrix
samples <- intersect(samples, colnames(count_matrix))

count_matrix <- count_matrix[, samples]
metadata <- metadata[samples, , drop = FALSE]

# Create DESeq2 data
dds <- DESeqDataSetFromMatrix(countData = as.matrix(count_matrix), colData = metadata, design = ~ Diagnosis)

# Run DESeq2
dds <- DESeq(dds)

# Extract results
res <- results(dds, contrast = c("Diagnosis", "Hp", "Atr"))

# Save results
write.csv(res, "STAR.DESeq2_results.csv")
"""

# Run DESeq2 inside Python
robjects.r(r_deseq_script)




R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: -- replacing outliers and refitting for 413 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

R[write to console]: estimating dispersions

R[write to console]: fitting model and testing



<rpy2.rinterface_lib.sexp.NULLType object at 0x1510060874d0> [0]

In [33]:
import rpy2.robjects as robjects

r_deseq_script = """
library(DESeq2)                                              

#Try the deseq analysis using the subread data
# Load count matrix
count_matrix <- read.delim("data/merged_counts_matrix.csv",  header = TRUE, sep = "\t", row.names=1, check.names=FALSE)
#count_matrix <- count_matrix[, -1]
metadata <- read.table("data/STAR_DE_analysis/STAR_metadata.txt", header = TRUE)
count_matrix <- as.matrix(count_matrix)
mode(count_matrix) <- "numeric"


rownames(metadata) <- metadata$Sample_ID

#create the factors from diagnosis
metadata$Diagnosis <- factor(metadata$Diagnosis, levels = c("Atr", "Hp"))

samples <- rownames(metadata)

# Keep only samples present in both metadata and count matrix
samples <- intersect(samples, colnames(count_matrix))

count_matrix <- count_matrix[, samples]
metadata <- metadata[samples, , drop = FALSE]

# Create DESeq2 data
dds <- DESeqDataSetFromMatrix(countData = as.matrix(count_matrix), colData = metadata, design = ~ Diagnosis)

# Run DESeq2
dds <- DESeq(dds)

# Extract results
res <- results(dds, contrast = c("Diagnosis", "Hp", "Atr"))

# Save results
write.csv(res, "STAR.DESeq2.subread_results.csv")
"""

# Run DESeq2 inside Python
robjects.r(r_deseq_script)




R[write to console]: converting counts to integer mode

R[write to console]: Error in DESeqDataSet(se, design = design, ignoreRank) : 
  all samples have 0 counts for all genes. check the counting script.



RRuntimeError: Error in DESeqDataSet(se, design = design, ignoreRank) : 
  all samples have 0 counts for all genes. check the counting script.


filter results using python

In [18]:
import pandas as pd
import statsmodels.stats.multitest as smm
DESeq2_STAR_results = pd.read_csv("STAR.DESeq2_results.csv") 

DESeq2_STAR_results = DESeq2_STAR_results.dropna(subset=["padj"]) #drop all NAN in padj column
significant, corrected_pvals = smm.fdrcorrection(DESeq2_STAR_results["padj"], alpha=0.05, method='indep', is_sorted=False)
DESeq2_STAR_results["FDR_corrected_padj"] = corrected_pvals
DESeq2_STAR_results["Significant"] = significant 

DESeq2_STAR_filtered_results = DESeq2_STAR_results.sort_values("FDR_corrected_padj", ascending=True) 
print(DESeq2_STAR_filtered_results.loc[DESeq2_STAR_filtered_results["padj"]<=0.05])
print(len(DESeq2_STAR_filtered_results[DESeq2_STAR_filtered_results["Significant"]==True]))

            Unnamed: 0     baseMean  log2FoldChange     lfcSE      stat  \
53956  ENSG00000211897  2344.407879       -4.478580  0.531624 -8.424334   
62320  ENSG00000163735   123.902293       -7.465867  0.888871 -8.399267   
38348  ENSG00000211893  2147.097202       -4.209961  0.515895 -8.160507   
11211  ENSG00000145113   766.820801       -5.530088  0.706498 -7.827464   
565    ENSG00000269404    65.987284       -5.324327  0.686340 -7.757569   
...                ...          ...             ...       ...       ...   
44565  ENSG00000198894   203.482200        0.402649  0.137751  2.923010   
34318  ENSG00000175449    18.735403        1.000771  0.342404  2.922780   
35466  ENSG00000254777    26.416927       -1.349131  0.461829 -2.921280   
75771  ENSG00000007933    68.438163       -1.241131  0.424991 -2.920371   
37991  ENSG00000247516   144.270115        0.801186  0.274420  2.919557   

             pvalue          padj  FDR_corrected_padj  Significant  
53956  3.628069e-17  6.590410e